In [50]:
#Exploring the data

!pip install ydata_profiling


In [51]:
#Step 1:Importing necessary libraries
#-pandas and numpy for data manipulation
#-sklearn for machine learning tasks
#-ydata-profiling for generating data insights
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from ydata_profiling import ProfileReport
from sklearn.svm import SVR
from sklearn.metrics import r2_score, mean_squared_error

In [52]:
#Load the dataset
df = pd.read_csv('/content/Synthetic_Smartphone_Data.csv')

#Show the first class
df.head()

Brand  Model       OS 5G Support  Release Year  Battery Capacity  \
0    Apple  Pixel      iOS         No          2022              4402   
1  Samsung    Pro  Android        Yes          2019              4042   
2   Google   Nord  Android         No          2020              4738   
3   Xiaomi  Ultra  Android        Yes          2021              4817   
4  OnePlus   Note  Android         No          2020              3036   

   Screen Size   Carrier  Retail Price ($100)  
0     6.288087      AT&T             5.781120  
1     6.246696  T-Mobile             9.142798  
2     6.171450   Verizon             5.719867  
3     5.728010      AT&T            11.605708  
4     5.770253  T-Mobile             9.723708

In [53]:
#Step 3:Describe data
df.describe(include='all')

Brand Model       OS 5G Support  Release Year  Battery Capacity  \
count     501   501      501        501    501.000000        501.000000   
unique      5     5        2          2           NaN               NaN   
top     Apple  Nord  Android         No           NaN               NaN   
freq      101   101      400        251           NaN               NaN   
mean      NaN   NaN      NaN        NaN   2020.297405       3705.642715   
std       NaN   NaN      NaN        NaN      1.097895        857.061490   
min       NaN   NaN      NaN        NaN   2019.000000       1914.000000   
25%       NaN   NaN      NaN        NaN   2019.000000       3033.000000   
50%       NaN   NaN      NaN        NaN   2020.000000       3644.000000   
75%       NaN   NaN      NaN        NaN   2021.000000       4516.000000   
max       NaN   NaN      NaN        NaN   2022.000000       4998.000000   

        Screen Size   Carrier  Retail Price ($100)  
count    501.000000       500           500.000000  
unique          NaN         3                  NaN  
top             NaN  T-Mobile                  NaN  
freq            NaN       168                  NaN  
mean       5.976080       NaN             8.793044  
std        0.200512       NaN             0.412155  
min        5.635680       NaN             5.719867  
25%        5.817954       NaN             8.680127  
50%        5.972403       NaN             8.800877  
75%        6.151625       NaN             8.919627  
max        6.448446       NaN            11.606595

In [54]:
#Step 4:Check for missing values
df.isnull().sum()

Brand                  0
Model                  0
OS                     0
5G Support             0
Release Year           0
Battery Capacity       0
Screen Size            0
Carrier                1
Retail Price ($100)    1
dtype: int64

In [55]:
#Drop all the irrelevant columns I do not need
df.drop(['Carrier'],axis=1,inplace=True)

In [56]:
#Step 5:Filling in missing values
#For retail price
df['Retail Price ($100)'].fillna(df['Retail Price ($100)'].median(),inplace=True)


/tmp/ipython-input-56-935953943.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Retail Price ($100)'].fillna(df['Retail Price ($100)'].median(),inplace=True)


In [57]:
# Define a function to remove outliers using IQR
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_cleaned = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)].copy()
    return df_cleaned

# Applying outlier removal to 'Retail Price ($100)','Battery Capacity
df = remove_outliers(df, 'Retail Price ($100)')


In [58]:
#Step 8:Encoding categorical features
#-Using LabelEncoder to convert categorical columns(Brand,Model,OS,5G_support)
#-SVM requires numerical inputs,so encoding is necessary for model compatibility
le=LabelEncoder()
categorical_Features=['Brand','Model','OS','5G Support']
for feature in categorical_Features:
  df[feature]=le.fit_transform(df[feature])

In [59]:
#Finds shape of data
df.shape

(472, 8)

***2.Building an SVM model***

In [60]:
#Step 1:Selecting target feature and variables
#-Target:'Retail Price ($100)' (price in hundreds of dollars)
#-Features:All other columns
#-Ensures the model uses all relevant information for prediction
X=df.drop('Retail Price ($100)',axis=1)
y=df['Retail Price ($100)']

In [61]:
#Step 2:Splitting dataset into training and test sets
#-Using 80% for training and 20 % for testing with a fixed random state
#-Ensures consistent evaluation and sufficient data for training and testing
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [62]:
#Step 3:Scaling features
#-Standardizing feauters
#-SVM is sensitive to feature scaler,so scaling improves model convergency and accuracy
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [63]:
#Step 4:Building and training SVM model
svm_model=SVR(kernel='rbf')
svm_model.fit(X_train,y_train)

SVR()

In [64]:
#Step 5:Build and train SVR with polynomial kernel
svm_poly_model=SVR(kernel='poly',degree=3)
svm_poly_model.fit(X_train,y_train)

SVR(kernel='poly')

In [69]:
#Step 6:Assess both the rbf and poly models performance
y_pred_rbf=svm_model.predict(X_test)
y_pred_poly=svm_poly_model.predict(X_test)

#Calculate R2 scores for both models
r2_rbf=r2_score(y_test,y_pred_rbf)
r2_poly=r2_score(y_test,y_pred_poly)

#Calculate RSME scores for both models
rmse_rbf=np.sqrt(mean_squared_error(y_test,y_pred_rbf))
rmse_poly=np.sqrt(mean_squared_error(y_test,y_pred_poly))

#Print the results
print("R2 Score (RBF Kernel):",r2_rbf)
print("RMSE Score (RBF Kernel):",rmse_rbf)
print("R2 Score (Polynomial Kernel):",r2_poly)
print("RMSE Score (Polynomial Kernel):",rmse_poly)
print('/n......................\n')

R2 Score (RBF Kernel): 0.6123568231361004
RMSE Score (RBF Kernel): 0.09463434939008894
R2 Score (Polynomial Kernel): 0.6408217803161684
RMSE Score (Polynomial Kernel): 0.09109357058287311
/n......................



In [68]:
# Step 4:Building and training SVM model
svm_model=SVR(kernel='rbf')
svm_model.fit(X_train,y_train)

SVR()

In [67]:
# Step 5: Build and train SVR with polynomial kernel
svm_poly_model=SVR(kernel='poly',degree=3)
svm_poly_model.fit(X_train,y_train)

SVR(kernel='poly')